In [1]:
import qiskit
import cirq
import braket
from braket.circuits import Circuit as BraketCircuit
from braket.circuits.gate import Gate as BraketGate
from braket.circuits.instruction import Instruction as BraketInstruction
from braket.circuits.unitary_calculation import calculate_unitary
from qbraid.transpiler.transpiler import qbraid_wrapper
import numpy as np

### cirq to qiskit transpiler qubit indexing / matrix equivalence problem explained

In [2]:
# construct a regular qiskit circuit
qiskit_circ = qiskit.QuantumCircuit(4)

qiskit_circ.h([0, 1, 2, 3])
qiskit_circ.x([0, 1])
qiskit_circ.y(2)
qiskit_circ.z(3)
qiskit_circ.s(0)
qiskit_circ.sdg(1)
qiskit_circ.t(2)
qiskit_circ.tdg(3)
qiskit_circ.rx(np.pi/4, 0)
qiskit_circ.ry(np.pi/2, 1)
qiskit_circ.rz(3*np.pi/4, 2)
qiskit_circ.p(np.pi/8, 3)
qiskit_circ.sx(0)
qiskit_circ.sxdg(1)
qiskit_circ.iswap(2, 3)
qiskit_circ.swap([0, 1], [2, 3])
qiskit_circ.cx(0, 1)
qiskit_circ.cp(np.pi/4, 2, 3)

qiskit_circ.draw()

┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ RX(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ SDG ├┤ RY(π/2) ├──┤ √XDG ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ RZ(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  ISWAP │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ TDG ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘

In [3]:
# Now let's make the same cirq circuit
cirq_circuit = cirq.Circuit()

# here the qubit ordering is normal, from 0 up to 3
q0 = cirq.LineQubit(0)
q1 = cirq.LineQubit(1)
q2 = cirq.LineQubit(2)
q3 = cirq.LineQubit(3)

cirq_gates = [
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.H(q3),
    cirq.X(q0),
    cirq.X(q1),
    cirq.Y(q2),
    cirq.Z(q3),
    cirq.S(q0),
    cirq.ZPowGate(exponent=-0.5)(q1),
    cirq.T(q2),
    cirq.ZPowGate(exponent=-0.25)(q3),
    cirq.Rx(rads=np.pi / 4)(q0),
    cirq.Ry(rads=np.pi / 2)(q1),
    cirq.Rz(rads=3 * np.pi / 4)(q2),
    cirq.ZPowGate(exponent=1 / 8)(q3),
    cirq.XPowGate(exponent=0.5)(q0),
    cirq.XPowGate(exponent=-0.5)(q1),
    cirq.ISWAP(q2, q3),
    cirq.SWAP(q0, q2),
    cirq.SWAP(q1, q3),
    cirq.CNOT(q0, q1),
    cirq.CZPowGate(exponent=0.25)(q2, q3),
]

for gate in cirq_gates:
    cirq_circuit.append(gate)

# visually, this circuit appears the same as the original qiskit circuit
print(cirq_circuit)

                                          ┌──┐
0: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                           │     │
1: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           ││
2: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@────────
                                 │          │    │
3: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@^0.25───
                                          └──┘


In [4]:
# calculate unitaries
qiskit_unitary = qiskit.quantum_info.Operator(qiskit_circ).data
cirq_unitary = cirq_circuit.unitary()

In [5]:
# However, the matrix representations of these circuits are NOT equivalent
np.allclose(qiskit_unitary, cirq_unitary)

False

In [6]:
# Now let's construct an identical cirq circuit, but with the qubits "flipped"
cirq_circuit_flipped = cirq.Circuit()

# notice the qubits are numbered from 3 down to 0 
q0 = cirq.LineQubit(3)
q1 = cirq.LineQubit(2)
q2 = cirq.LineQubit(1)
q3 = cirq.LineQubit(0)

cirq_gates = [
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.H(q3),
    cirq.X(q0),
    cirq.X(q1),
    cirq.Y(q2),
    cirq.Z(q3),
    cirq.S(q0),
    cirq.ZPowGate(exponent=-0.5)(q1),
    cirq.T(q2),
    cirq.ZPowGate(exponent=-0.25)(q3),
    cirq.Rx(rads=np.pi / 4)(q0),
    cirq.Ry(rads=np.pi / 2)(q1),
    cirq.Rz(rads=3 * np.pi / 4)(q2),
    cirq.ZPowGate(exponent=1 / 8)(q3),
    cirq.XPowGate(exponent=0.5)(q0),
    cirq.XPowGate(exponent=-0.5)(q1),
    cirq.ISWAP(q2, q3),
    cirq.SWAP(q0, q2),
    cirq.SWAP(q1, q3),
    cirq.CNOT(q0, q1),
    cirq.CZPowGate(exponent=0.25)(q2, q3),
]

for gate in cirq_gates:
    cirq_circuit_flipped.append(gate)

# same circuit as qiskit above, but with qubits flipped
print(cirq_circuit_flipped)

                                          ┌──┐
0: ───H───Z───T^-1───Z^(1/8)─────iSwap──────×────@────────
                                 │          │    │
1: ───H───Y───T──────Rz(0.75π)───iSwap─────×┼────@^0.25───
                                           ││
2: ───H───X───S^-1───Ry(0.5π)────X^-0.5────┼×────X────────
                                           │     │
3: ───H───X───S──────Rx(0.25π)───X^0.5─────×─────@────────
                                          └──┘


In [7]:
# calculate the matrix representation of the "flipped" circuit
cirq_unitary_flipped = cirq_circuit_flipped.unitary()

In [8]:
# The matrix representations of qiskit_circuit and cirq_circuit_flipped are equivalent
np.allclose(qiskit_unitary, cirq_unitary_flipped)

True

In [9]:
# Transpiling the original qiskit circuit to cirq should result in a circuit that looks the same and has the same 
# matrix representation as as the non-flipped cirq circuit
qbraid_circuit_0 = qbraid_wrapper(qiskit_circ)
cirq_circuit_transpiled = qbraid_circuit_0.transpile("cirq")
cirq_unitary_transpiled = cirq_circuit_transpiled.unitary()

print(cirq_circuit_transpiled)

                                      ┌──┐
0: ───H───X───S───Rx(0.25π)───U────────×─────@────────
                                       │     │
1: ───H───X───U───Ry(0.5π)────U────────┼×────X────────
                                       ││
2: ───H───Y───T───Rz(0.75π)───iSwap────×┼────@────────
                              │         │    │
3: ───H───Z───U───Z^(1/8)─────iSwap─────×────@^0.25───
                                      └──┘


In [10]:
# This returns True, so transpiling a qiskit circuit to cirq works as expected
np.allclose(cirq_unitary_transpiled, cirq_unitary)

True

In [11]:
# Now let's try transpiling a cirq circuit to qiskit. Transpiling the non-flipped cirq circuit to qiskit 
# should result in the same qiskit circuit and matrix representation as the original. 
qbraid_circuit_1 = qbraid_wrapper(cirq_circuit)
qiskit_circuit_transpiled = qbraid_circuit_1.transpile("qiskit")
qiskit_unitary_transpiled = qiskit.quantum_info.Operator(qiskit_circuit_transpiled).data

qiskit_circuit_transpiled.draw()

┌───┐┌───┐   ┌───┐   ┌─────────┐┌─────────┐                  
q0_0: ┤ H ├┤ X ├───┤ S ├───┤ RX(π/4) ├┤ Unitary ├─X─────────■──────
      ├───┤├───┤┌──┴───┴──┐├─────────┤├─────────┤ │       ┌─┴─┐    
q0_1: ┤ H ├┤ X ├┤ P(-π/2) ├┤ Unitary ├┤ Unitary ├─┼──X────┤ X ├────
      ├───┤├───┤└──┬───┬──┘├─────────┤└┬────────┤ │  │ ┌──┴───┴───┐
q0_2: ┤ H ├┤ Y ├───┤ T ├───┤ P(3π/4) ├─┤0       ├─X──┼─┤0         ├
      ├───┤├───┤┌──┴───┴──┐└┬────────┤ │  ISWAP │    │ │  Unitary │
q0_3: ┤ H ├┤ Z ├┤ P(-π/4) ├─┤ P(π/8) ├─┤1       ├────X─┤1         ├
      └───┘└───┘└─────────┘ └────────┘ └────────┘      └──────────┘

In [12]:
# But the matrix representation is not equivalent to the original
np.allclose(qiskit_unitary_transpiled, qiskit_unitary)

False

In [13]:
braket_circuit = BraketCircuit()

instructions = [
    BraketInstruction(BraketGate.H(), 0),
    BraketInstruction(BraketGate.H(), 1),
    BraketInstruction(BraketGate.H(), 2),
    BraketInstruction(BraketGate.H(), 3),
    BraketInstruction(BraketGate.X(), 0),
    BraketInstruction(BraketGate.X(), 1),
    BraketInstruction(BraketGate.Y(), 2),
    BraketInstruction(BraketGate.Z(), 3),
    BraketInstruction(BraketGate.S(), 0),
    BraketInstruction(BraketGate.Si(), 1),
    BraketInstruction(BraketGate.T(), 2),
    BraketInstruction(BraketGate.Ti(), 3),
    BraketInstruction(BraketGate.Rx(np.pi/4), 0),
    BraketInstruction(BraketGate.Ry(np.pi/2), 1),
    BraketInstruction(BraketGate.Rz(3*np.pi/4), 2),
    BraketInstruction(BraketGate.PhaseShift(np.pi/8), 3),
    BraketInstruction(BraketGate.V(), 0),
    BraketInstruction(BraketGate.Vi(), 1),
    BraketInstruction(BraketGate.ISwap(), [2, 3]),
    BraketInstruction(BraketGate.Swap(), [0, 2]),
    BraketInstruction(BraketGate.Swap(), [1, 3]),
    BraketInstruction(BraketGate.CNot(), [0, 1]),
    BraketInstruction(BraketGate.CPhaseShift(np.pi/4), [2, 3]),
]

for inst in instructions:
    braket_circuit.add_instruction(inst)

print(braket_circuit)

T  : |0|1|2 |     3      |  4  |    5    |     6      |
                                                       
q0 : -H-X-S--Rx(0.785)----V-----SWAP------C------------
                                |         |            
q1 : -H-X-Si-Ry(1.57)-----Vi----|----SWAP-X------------
                                |    |                 
q2 : -H-Y-T--Rz(2.36)-----ISWAP-SWAP-|----C------------
                          |          |    |            
q3 : -H-Z-Ti-PHASE(0.393)-ISWAP------SWAP-PHASE(0.785)-

T  : |0|1|2 |     3      |  4  |    5    |     6      |


In [14]:
braket_unitary = calculate_unitary(4, instructions)

In [15]:
np.allclose(braket_unitary, qiskit_unitary)

True